<img src="https://cdn-assets-cloud.frontify.com/local/frontify/eyJwYXRoIjoiXC9wdWJsaWNcL3VwbG9hZFwvc2NyZWVuc1wvMTk3OTA0XC80M2ZmNTdhYjc4OTdlZjUzY2IzMWUwNGU0MTVjZTY2NC0xNTYyMTAzMDk0LnBuZyJ9:frontify:7CTV2DtJsWvlctEUEyFK36JoXsZuVtHssMaDED6O5z0" width='150' />

# VECTOR SEARCH - RETRIEVAL AUGMENTED GENERATION

__How to use this notebook__

1. Run the code cell below and paste the following into the password widgets:
    1. Your Atlas cluster URI (w/ read/write permissions)
    1. Your OpenAI API key (used to create embeddings and pose questions to an LLM)
1. If desired, use a custom PDF URL and ask custom questions at the end
1. Present the notebook via the "Enter/Exit RISE Slideshow" toolbar button (looks like a bar chart)
    1. Put your browser into full-screen mode for best results
    1. To advance a cell without executing, use "Space"
    1. To execute the current cell, use "Shift-Enter"

In [4]:
import ipywidgets as widgets
import os

mongodb_uri_widget = widgets.Password(
    description='Your Atlas URI:',
    disabled=False,
    style=dict(description_width='125px')
)

openai_api_key_widget = widgets.Password(
    description='Your OpenAI API key:',
    disabled=False,
    style=dict(description_width='125px')
)

display(mongodb_uri_widget)
display(openai_api_key_widget)

ModuleNotFoundError: No module named 'ipywidgets'

# Retrieval Augmented Generation
### Using MongoDB Atlas, OpenAI and LangChain

In [145]:
from IPython.display import IFrame

PDF_URI = "https://rcg-demo1.s3.amazonaws.com/Top+Things+to+Do+at+Perfect+Day+at+Cococay+_+Royal+Caribbean+Cruises.pdf"
IFrame(PDF_URI, width=1280, height=500)

# Get connection to MongoDB Atlas

In [156]:
from pymongo import MongoClient
import os

mongo_db_name = 'rag_demo'
mongo_coll_name = 'content'

mongo_client = MongoClient(mongodb_uri_widget.value)
mongo_coll = mongo_client[mongo_db_name][mongo_coll_name]
mongo_db_and_coll_path = '{}.{}'.format(mongo_db_name, mongo_coll_name)

doc_count = mongo_coll.count_documents({})
'{} document count is {:,}'.format(mongo_db_and_coll_path, doc_count)

'rag_demo.content document count is 22'

In [157]:
# Delete existing documents -- run before demo
mongo_coll.delete_many({})

# Select embeddings/transformer model

In [158]:
from langchain.embeddings import OpenAIEmbeddings

embeddings_model = OpenAIEmbeddings(
    model='text-embedding-ada-002',
    openai_api_key=openai_api_key_widget.value
)

print('Max token length is 8,191')

Max token length is 8,191


# Split PDF into chunks

In [159]:
from langchain.document_loaders import PyPDFLoader

loader = PyPDFLoader(PDF_URI)
chunked_docs = loader.load_and_split()

'PDF has resulted in {:,} chunks'.format(len(chunked_docs))

'PDF has resulted in 22 chunks'

In [160]:
biggest_chunk_length = max(len(chunk.page_content.split()) for chunk in chunked_docs)
'The biggest chunk contains {:,} words'.format(biggest_chunk_length)

'The biggest chunk contains 180 words'

# Create vectors and add to MongoDB Atlas

In [161]:
from langchain.vectorstores import MongoDBAtlasVectorSearch

vector_db = MongoDBAtlasVectorSearch.from_documents(
    chunked_docs,
    embeddings_model,
    collection=mongo_coll
)

In [162]:
doc_count = mongo_coll.count_documents({})
'MongoDB document count in {} is {:,}'.format(mongo_db_and_coll_path, doc_count)

'MongoDB document count in rag_demo.content is 22'

# Create MongoDB Atlas vector search index

In [94]:
from pymongo.errors import OperationFailure
import inspect

mongo_index_def = {
    'name': 'rag_demo_index',
    'definition': {
        'mappings': {
            'dynamic': True,
            'fields': {
                'embedding': {
                    'type': 'knnVector',
                    'dimensions': 1536,
                    'similarity': 'cosine'
                }
            }
        }
    }
}

try:
    mongo_coll.create_search_index(mongo_index_def)
    print('Search index is building')
except OperationFailure as e:
    print(e.details['codeName'])

TypeError: 'Collection' object is not callable. If you meant to call the 'create_search_index' method on a 'Collection' object it is failing because no such method exists.

# Create a LangChain handle for the vector search index

In [163]:
vector_db = MongoDBAtlasVectorSearch.from_connection_string(
    mongodb_uri_widget.value,
    mongo_db_and_coll_path,
    embeddings_model,
    index_name='rag_demo_index'
)

# Setup question function

In [164]:
from langchain.chains import ConversationalRetrievalChain
from langchain.schema.document import Document
from langchain.chat_models import ChatOpenAI

llm_model = ChatOpenAI(
    model_name='gpt-3.5-turbo',
    temperature=0.0,
    openai_api_key=openai_api_key_widget.value
)

pdf_qa = ConversationalRetrievalChain.from_llm(
    llm_model,
    vector_db.as_retriever(),
    return_source_documents=True
)

def ask_question(question):
    result = pdf_qa({'question': 'Answer only if this information is available in the source document - ' + question, 'chat_history': []})
    print("Answer:{}\n".format(result.get('answer')))
    print('Chunks from Atlas Vector Search used for context:')
    
    for chunk in result.get('source_documents'):
        id = chunk.metadata['_id']
        page = chunk.metadata['page']
        print('ObjectId({}) | page {:,}'.format(id, page))

In [109]:
# from langchain.llms import OpenAI
#
# llm_model = OpenAI(
#     model_name='text-davinci-003',
#     temperature=0.0,
#     openai_api_key=os.environ['OPENAI_API_KEY']
# )

# from langchain.chat_models import ChatOpenAI
#
# llm_model = ChatOpenAI(
#     model_name='gpt-3.5-turbo',
#     temperature=0.0,
#     openai_api_key=os.environ['OPENAI_API_KEY']
# )

# Start asking questions

In [165]:
ask_question("What is Cococay?")

Answer:Cococay is an island destination in The Bahamas that is owned and operated by Royal Caribbean Cruises. It is a private island exclusively for Royal Caribbean guests and offers a variety of attractions, amenities, and activities for visitors to enjoy.

Chunks from Atlas Vector Search used for context:
ObjectId(6568f41acee9d4567406c437) | page 10
ObjectId(6568f41acee9d4567406c441) | page 20
ObjectId(6568f41acee9d4567406c440) | page 19
ObjectId(6568f41acee9d4567406c442) | page 21


In [166]:
ask_question("Tell me all the family activities we can do at COCOCAY?")

Answer:The source document mentions that Perfect Day at CocoCay is considered one of the best family-friendly cruise destinations. It offers many activities and attractions that both kids and adults can enjoy together. Some of the family activities mentioned include volleyball, beachside basketball, snorkeling, paddleboarding, and even ping-pong at South Beach. Additionally, there are complimentary access, amenities, and activities at attractions like Oasis Lagoon, Chill Island, and South Beach.

Chunks from Atlas Vector Search used for context:
ObjectId(6568f41acee9d4567406c437) | page 10
ObjectId(6568f41acee9d4567406c432) | page 5
ObjectId(6568f41acee9d4567406c433) | page 6
ObjectId(6568f41acee9d4567406c42d) | page 0


In [167]:
ask_question("What is MongoDB")

Answer:The source document does not provide information about what MongoDB is.

Chunks from Atlas Vector Search used for context:
ObjectId(6568f41acee9d4567406c442) | page 21
ObjectId(6568f41acee9d4567406c437) | page 10
ObjectId(6568f41acee9d4567406c441) | page 20
ObjectId(6568f41acee9d4567406c440) | page 19


In [168]:
# Use this cell to show that the majority of time spent waiting is due to the LLM, not Atlas Vector Search

import time

search_vector = embeddings_model.embed_query("How should I optimize query performance?")

before_time = time.perf_counter()
cursor = mongo_coll.aggregate([
    {
        "$vectorSearch": {
            "index": "rag_demo_index",
            "path": "embedding",
            "queryVector": search_vector,
            "numCandidates": 100,
            "limit": 4
        }
    },
    {
        "$project": {
            "_id": 1,
            "page": 1,
        }
    }
])
vector_search_ms = int((time.perf_counter() - before_time) * 1_000)
print('Atlas Vector Search roundtrip took {} ms'.format(vector_search_ms))
list(cursor)

Atlas Vector Search roundtrip took 25 ms


[{'_id': ObjectId('6568f41acee9d4567406c437'), 'page': 10},
 {'_id': ObjectId('6568f41acee9d4567406c441'), 'page': 20},
 {'_id': ObjectId('6568f41acee9d4567406c434'), 'page': 7},
 {'_id': ObjectId('6568f41acee9d4567406c435'), 'page': 8}]